# LAMMPS Tutorials 08. Simulate a single polymer chain!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

Thanks to Dmitry Zhuk for writing the first instantiation of this tutorial!

***
## Abstract:
<a id="Sec1"></a>

In this tutorial, a molecular dynamics simulation in LAMMPS is used to show what happens to a polymer chain at a certain temperature after some time. The chain's movement is caused by a molecular forces between atoms in the chain and by temperature of the chain. These factors are modeled in LAMMPS to show the behavior of this polymer chain. The chain was previously prepared in MATLAB. It contains 100 atoms with bound between neighbor atoms. During the simulation, the chain was equilibrated and fluctuates at temperature. After that the chain was minimized to find it's minimal energy condition. The tutorial explains basic LAMMPS commands and shows how to make a movie using AtomEye and ImageJ. 

<table><tr><td><img src='https://icme.hpc.msstate.edu/mediawiki/images/8/89/Equ_plus_Min.gif' width="300"><center><br>Equilibration process followed by minimization</center></td></tr></table>

***
## Complete the First Tutorials

If you have not done so already, you may want to complete the first tutorials available [here](https://github.com/mrkllntschpp/lammps-tutorials). 

***
## Step 1: Download Polymer Configuration file

The polymer chain of 100 atoms was initially written in MATLAB. The atom's Z coordinate does not vary much, all of them are within 2 Å. The distance between atoms is about 1.5 Å. Essentially, the chain goes from left upper to right lower corner of the box. 

The data file is shown below and available for download here: https://icme.hpc.msstate.edu/mediawiki/images/e/e1/PE_cl100.txt

Just in case you are interested in what it looks like:

```
# Model for PE

     100     atoms
      99     bonds
      98     angles
      97     dihedrals

         1     atom types
         1     bond types
         1     angle types
         1     dihedral types

    0.0000   158.5000 xlo xhi
    0.0000   158.5000 ylo yhi
    0.0000   100.0000 zlo zhi

Masses

         1          14.02

Atoms

         1         1         1    5.6240    5.3279   51.6059
         2         1         1    7.4995    7.4810   50.2541
         3         1         1    8.2322    8.0236   51.2149
         4         1         1    9.6108    9.9075   51.7682
         5         1         1   11.5481   11.3690   50.4167
         6         1         1   12.9409   13.4562   50.2481
         7         1         1   14.4708   14.8569   50.0868
         8         1         1   16.1916   16.4790   50.5665
         9         1         1   17.1338   17.6853   51.8189
        10         1         1   19.1109   19.4000   50.3869

...

Bonds

         1         1         1         2
         2         1         2         3
         3         1         3         4
         4         1         4         5
         5         1         5         6
         6         1         6         7
         7         1         7         8
         8         1         8         9
         9         1         9        10
        10         1        10        11
 
...

Angles

         1         1         1         2         3
         2         1         2         3         4
         3         1         3         4         5
         4         1         4         5         6
         5         1         5         6         7
         6         1         6         7         8
         7         1         7         8         9
         8         1         8         9        10

...

Dihedrals

         1         1         1         2         3         4
         2         1         2         3         4         5
         3         1         3         4         5         6
         4         1         4         5         6         7
         5         1         5         6         7         8
         6         1         6         7         8         9
         7         1         7         8         9        10
```

Here, the first section defines the numbers of atoms (100), bonds(99), angles (98), and dihedrals (97). Then the number of atom types, bond types, angle types, and dihedral types (each 1 type for simplicity).   Then comes the simulation cell sizes in the x, y, and z directions (denoted by a `lo`(w) and 'hi'(gh) value in each dimension). 

Then comes the explicit description for the atoms and the connections for the bonds, angles, and dihedrals. In the Atom section, the first column is the atom number, then comes the atom type and a column with information not used in this tutorial (read the description in LAMMPS help). The last three columns are each atom's x, y and z coordinates, correspondingly. 

The Bond section is intuitive.  Each bond has a number referring to it (column 1 - there are 99 of them), the bond type is used to relate it to a potential function (column 2 - only type 1), and they each connect two atoms, which were labeled in the section just above that (columns 3 and 4).  

The Angle section lists the angles between atoms. The first column is the angle number, the second column is the angle ID, then comes the atom numbers between which the angle is defined. Example of the distribution of the atoms can be seen on the picture below.  Note that this picture is defined by the following text above:

`         7         1         7         8         9` (angle ID 7, angle type 1, between atoms 7, 8, and 9)

The Dihedral angle section is just a little bit more complicated. To define a dihedral angle, four atoms are needed. The syntax is similar to the angles section. The only difference is that one more atom is needed to define a dihedral. Basically, the dihedral angle is the angle between the planes formed by 2 groups of 3 neighbor atoms. An example of the dihedral angle is shown below. Note that this picture is defined by the following text above:

`         4         1         4         5         6         7` (dihedral ID 4, dihedral type 1, between atoms 4, 5, 6, and 7)

<table><tr><td><img src='https://icme.hpc.msstate.edu/mediawiki/images/f/ff/Atom_angle.jpg' width="300"><center><br>Angles between atoms defined in LAMMPS</center></td><td><img src='https://icme.hpc.msstate.edu/mediawiki/images/a/a8/Dihedral.jpg' width="300"><center><br>Dihedral angles between atoms defined in LAMMPS</center></td></tr></table>

## Step 2: Create the LAMMPS input script 

This input script was run using the January 2020 version of LAMMPS. Changes in some commands in more recent versions may require revision of the input script. This script runs the simulation with the previously generated polymer chain file, which is fed to LAMMPS through the `read_data` command.  To run this script, store it in `in.deform_polymer_chain.txt`and use 

`lmp_exe < in.deform_polymer_chain.txt` 

where `lmp_exe` refers to the LAMMPS executable. 

In [1]:
%%writefile in.deform_polymer_chain.txt
######################################
# LAMMPS INPUT SCRIPT
# Polymer Chain Tutorial
# Mark Tschopp
# The methodology outlined here follows that from Hossain, Tschopp, et al. 2010, Polymer.
# The following script requires a LAMMPS data file containing the coordinates and 
# appropriate bond/angle/dihedral lists for each united atom.                                      
# syntax: lmp_exe -in in.deform_polymer_chain.txt

######################################
# VARIABLES
variable fname index PE_cl100.txt
variable simname index PE_cl100

######################################
# INITIALIZATION
units real
boundary f f f
atom_style molecular
log log.${simname}.txt
read_data ${fname}

######################################
# DEFINE INTERATOMIC POTENTIAL
# Dreiding potential
neighbor 0.4 bin
neigh_modify every 10 one 10000
bond_style      harmonic
bond_coeff 1 350 1.53
angle_style     harmonic
angle_coeff 1 60 109.5
dihedral_style multi/harmonic
dihedral_coeff 1 1.73 -4.49 0.776 6.99 0.0
pair_style lj/cut 10.5
pair_coeff 1 1 0.112 4.01 10.5

######################################
# DEFINE COMPUTES 
compute csym all centro/atom fcc
compute peratom all pe/atom 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

#####################################################
# EQUILIBRATION
# Langevin dynamics at 500 K

velocity all create 500.0 1231
fix 1 all nve/limit 0.05
fix 2 all langevin 500.0 500.0 10.0 904297
thermo_style custom step temp 
thermo          100
timestep 1
run 10000
unfix 1
unfix 2
write_restart restart.${simname}.dreiding1

#####################################################
# MINIMIZATION

dump 1 all cfg 6 dump.PE_cl100_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz
reset_timestep 0 
fix 1 all nvt temp 500.0 500.0 100.0
thermo 20 
thermo_style custom step pe lx ly lz press pxx pyy pzz c_eatoms 
min_style cg
minimize 1e-15 1e-15 1000 1000 

#####################################################
print "All done"

Writing in.deform_polymer_chain.txt


## United atom potential

Notice that we didn't have to download a potential file for this simulation.  All of the potential parameters necessary are programmed into those few lines within the LAMMPS script (see below).  The concept of a united atom is that each individual "united" atom actually represents multiple atoms, in this case the carbon and two hydrogen atoms from a polyethylene chain (i.e., cl100 represents C$_{100}$H$_{202}$). 

<br>
<div class="alert alert-block alert-info">
neighbor 0.4 bin <br>
neigh_modify every 10 one 10000 <br>
bond_style      harmonic <br>
bond_coeff 1 350 1.53 <br>
angle_style     harmonic <br>
angle_coeff 1 60 109.5 <br>
dihedral_style multi/harmonic <br>
dihedral_coeff 1 1.73 -4.49 0.776 6.99 0.0 <br>
pair_style lj/cut 10.5 <br>
pair_coeff 1 1 0.112 4.01 10.5
</div>

where the first number after the `pair_coeff`, `bond_coeff`, `angle_coeff`, and `dihedral_coeff` are the "IDs" from column 2 of the data file in Step 1, thereby identifying the potential for each of these components.

## Step 3: Run the LAMMPS Simulation

Now run the simulation as we have done before using the syntax 

`lmp_exe < in.deform_polymer_chain.txt`.  

On my computer, the 24Jan2020 LAMMPS executable is stored in the `C:\Program Files\LAMMPS 64-bit 24Jan2020\bin\` folder and is named `lmp_serial.exe`.  

The `log.PE_cl100.txt` file should look like the output below. Whoa! 10,000 timesteps...  that must take a while. Nope. The simulation took less than a second on my computer (100,000 timesteps only took about 3 seconds).  Remember that this is only 100 atoms with a very simple potential.

In [2]:
import time
start_time = time.time()
!lmp_serial < in.deform_polymer_chain.txt
print("--- %s seconds ---" % (time.time() - start_time))

LAMMPS (3 Mar 2020)
Reading data file ...
  orthogonal box = (0 0 0) to (158.5 158.5 100)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  100 atoms
  scanning bonds ...
  1 = max bonds/atom
  scanning angles ...
  1 = max angles/atom
  scanning dihedrals ...
  1 = max dihedrals/atom
  reading bonds ...
  99 bonds
  reading angles ...
  98 angles
  reading dihedrals ...
  97 dihedrals
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:   0          0          0         
  special bond factors coul: 0          0          0         
  2 = max # of 1-2 neighbors
  2 = max # of 1-3 neighbors
  4 = max # of 1-4 neighbors
  6 = max # of special neighbors
  special bonds CPU = 0.000180006 secs
  read_data CPU = 0.00569391 secs
Neighbor list info ...
  update every 10 steps, delay 10 steps, check yes
  max neighbors/atom: 10000, page size: 100000
  master list distance cutoff = 10.9
  ghost atom cutoff = 10.9
  binsize = 5.45, bins = 30 30 19
  2 neighbor lists, perpetual/occasi

OK, the dump files should be able to be used to examine what is happening during the simulation.  Notice that the temperature jumps around quite a bit, a function of the small number of (unconstrained) atoms and the high temperature.  In the minimization stage, the energy starts very high and is rapidly reduced to a minimum energy, i.e., the minimum energy state at 0 K.

<table><tr><td><img src='https://icme.hpc.msstate.edu/mediawiki/images/8/89/Equ_plus_Min.gif' width="300"><center><br>Equilibration process followed by minimization</center></td></tr></table>


***
## LAMMPS Input Script Explained

In this section, we will break down what LAMMPS is doing. The easy way to do this on your own is to consult the LAMMPS manual for each command or go to the Internet LAMMPS manual, *i.e.*, at https://lammps.sandia.gov

In general, this script does equilibration and minimization to the polymer chain. Polymer chain data file named `PE_cl100.txt` should be in the present working directory. The line `dump 1 all cfg 6 dump.PE_cl100_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz` is used to output information during the simulation; this can be moved before the equilibration part of the script to output the process of equilibration. Please note that you need to change the time step for the dump command so it won't give too many or too few dump files. It may be reasonable to choose an `every n timesteps` number to correspond to roughly 100 snapshots. 

<br>
<div class="alert alert-block alert-info">
###################################### <br>
# VARIABLES <br>
variable fname index PE_cl100.txt <br>
variable simname index PE_cl100 <br> <br>
###################################### <br>
# INITIALIZATION <br>
units real <br>
boundary f f f <br>
atom_style molecular <br>
log log.\${simname}.txt <br>
read_data \${fname}
</div>

This part is just opens the data file, defines the boundary conditions, units, logfile's name, etc. 

<br>
<div class="alert alert-block alert-info">
###################################### <br>
# DEFINE INTERATOMIC POTENTIAL <br>
# Dreiding potential <br>
neighbor 0.4 bin <br>
neigh_modify every 10 one 10000 <br>
bond_style      harmonic <br>
bond_coeff 1 350 1.53 <br>
angle_style     harmonic <br>
angle_coeff 1 60 109.5 <br>
dihedral_style multi/harmonic <br>
dihedral_coeff 1 1.73 -4.49 0.776 6.99 0.0 <br>
pair_style lj/cut 10.5 <br>
pair_coeff 1 1 0.112 4.01 10.5
</div>

As mentioned earlier, this section gives the information about the potential functions/parameters to be used for the bonds, angles, dihedrals in a chain. Bond_style and bond_coeff defines the type on the force field between atoms and a magnitude of this fields. "1" here corresponds to the second column of the "Bonds" section of the data file. Thus, every atom pair with "1" in the second column has properties that correspond to that potential function during the simulation. The angles and dihedrals are similar. Angle_* and dihedral_* lines define the angles and dihedral angles between atoms in the polymer chain. Pair_* commands used to define pair potentials between atoms that are within a cutoff distance.

<br>
<div class="alert alert-block alert-info">
###################################### <br>
# DEFINE COMPUTES  <br>
compute csym all centro/atom fcc <br>
compute peratom all pe/atom  <br>
compute eng all pe/atom  <br>
compute eatoms all reduce sum c_eng
</div>

This commands are used to define which properties are to be calculated during the simulation. For example, `pe/atom` simply means to calculate the potential energy for each atom. Information about other possible properties to calculate can be found in the LAMMPS documentation.

<br>
<div class="alert alert-block alert-info">
##################################################### <br>
# EQUILIBRATION <br>
# Langevin dynamics at 500 K <br>
velocity all create 500.0 1231 <br>
fix 1 all nve/limit 0.05 <br>
fix 2 all langevin 500.0 500.0 10.0 904297 <br>
thermo_style custom step temp  <br>
thermo          100 <br>
timestep 1 <br>
run 10000 <br>
unfix 1 <br>
unfix 2 <br>
write_restart restart.${simname}.dreiding1 <br>
</div>

The equilibration step allow atoms in the polymer chain to move freely at a certain temperature. The `velocity` command instantiates the temperature for the various atoms within the chain. The `fix` command performs the check of the system every time step and updates the velocities and positions of the atoms. The `thermo` commands define the thermodynamic output during the simulation. The `run` command starts the simulation.  After the 10,000 timesteps are completed, the `unfix` commands remove the fixes; this is just a good book-keeping practice when the LAMMPS input scripts start to get a little more complicated.

<br>
<div class="alert alert-block alert-info">
##################################################### <br>
# MINIMIZATION <br>
dump 1 all cfg 6 dump.PE_cl100_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz <br>
reset_timestep 0  <br>
fix 1 all nvt temp 500.0 500.0 100.0 <br>
thermo 20  <br>
thermo_style custom step pe lx ly lz press pxx pyy pzz c_eatoms  <br>
min_style cg <br>
minimize 1e-15 1e-15 1000 1000
</div>

The other step in the program is the minimization. This process finds the minimum energy condition for this configuration. The parameters for minimize command include: (left to right) stopping tolerance for energy, stopping tolerance for force, max iterations of minimization routine and max number of force/energy evaluations, where stopping tolerance for energy is the first parameter and the max number of force/energy evaluations is the last one. 

***
## FAQs 

<br>
<div class="alert alert-danger">
    <strong>Question 1</strong>: None yet.  
</div>


***
## Links

* [Click here to open Tutorial 1](LAMMPS-Tutorials-01.ipynb)
* [Click here to open Tutorial 2](LAMMPS-Tutorials-02.ipynb)
* [Click here to open Tutorial 3](LAMMPS-Tutorials-03.ipynb)
* [Click here to open Tutorial 4](LAMMPS-Tutorials-04.ipynb)
* [Click here to open Tutorial 5](LAMMPS-Tutorials-05.ipynb)
* [Click here to open Tutorial 6](LAMMPS-Tutorials-06.ipynb)
* [Click here to open Tutorial 7](LAMMPS-Tutorials-07.ipynb)
* [Click here to open the next tutorial](LAMMPS-Tutorials-09.ipynb)

***
## References 

1. S. Plimpton, "Fast Parallel Algorithms for Short-Range Molecular Dynamics," J. Comp. Phys., 117, 1-19 (1995). 
1. Hossain, D., Tschopp, M.A. (corresponding author), Ward, D.K., Bouvard, J.L., Wang, P., Horstemeyer, M.F., "[Molecular dynamics simulations of deformation mechanisms of amorphous polyethylene](https://doi.org/10.1016/j.polymer.2010.10.009)," Polymer, 51 (2010) 6071-6083.